In [ ]:
!pip install langchain 
!pip install langchain_community

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from datetime import datetime
import unicodedata
import pickle
import os

In [34]:
def limpar_texto(texto):
  texto = texto.replace('[^\w\s]', '') # Remover caracteres especiais
  texto = texto.lower() # Converter para minúsculas
  return unicodedata.normalize('NFKD', texto).encode('ascii', errors='ignore').decode('utf-8') # Normalizes and removes accents from the text.

def get_name(texto):
  texto = texto[:len(texto)-8].replace("_", " ")
  return limpar_texto(texto)

# Adicionar nome do hotel aos chunks
def adicionar_nome_hotel_ao_chunk(docs, hotel_name):
    for doc in docs:
        doc.page_content = f"{hotel_name}, {doc.page_content}"
    return docs

In [35]:
# Diretório contendo os arquivos TXT
directory = 'TXT'

# Lista para armazenar os documentos
documents = []

# Iterar sobre cada arquivo TXT no diretório
for file_path in os.listdir(directory):
  if file_path.endswith('.txt'):
      with open(os.path.join(directory, file_path), 'r', encoding='utf-8') as file:
          # Ler o conteúdo do arquivo
          content = file.read()
          content = limpar_texto(content)
          hotel_name = get_name(file_path)
          # Criar um documento para o conteúdo completo do arquivo
          document = Document(content.strip(), metadata={"hotel_name": hotel_name})
          documents.append(document)

# Exibir o número total de documentos carregados
print(f"Total de documentos carregados: {len(documents)}")

Total de documentos carregados: 105


In [36]:
# Adaptar o text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(f"Total de Chunks: {len(docs)}")

Total de Chunks: 8208


In [37]:
docs_with_hotel_name = []
for document in documents:
    hotel_name = document.metadata["hotel_name"]
    chunks = text_splitter.split_documents([document])
    chunks_with_hotel_name = adicionar_nome_hotel_ao_chunk(chunks, hotel_name)
    docs_with_hotel_name.extend(chunks_with_hotel_name)


data_atual = datetime.now()
data_formatada = data_atual.strftime("%d-%m-%Y")
save_name = f"Chunks_RAG/documents_chunks_{data_formatada}.pkl"
# Salvar os documentos divididos com o nome do hotel
with open(save_name, 'wb') as f:
    
    pickle.dump(docs_with_hotel_name, f)